In [ ]:
import pandas as pd
import numpy as np
import requests as rq
import json

In [4]:
osc_bp = pd.read_csv('../data/scraping_results/osc_bp.csv')
len(osc_bp)

591

In [5]:
movies_nom = movies[movies['eligibleTitle'].isin(osc_bp['film'])]
len(movies_nom)
movies_nom

,eligibleTitle,searchYear,oscarsYear,runtimeMinutes,genres,winner
20,The Broadway Melody,1929,1929,100.0,"Drama,Musical,Romance",2
42,The Divorcee,1930,1930,84.0,"Drama,Romance",1
44,The Love Parade,1929,1930,107.0,"Comedy,Musical,Romance",1
69,Five Star Final,1931,1932,89.0,"Crime,Drama",1
77,One Hour with You,1932,1932,80.0,"Comedy,Musical,Romance",1
...,...,...,...,...,...,...
6266,La La Land,2016,2016,128.0,"Comedy,Drama,Music",1
6404,Get Out,2017,2017,104.0,"Horror,Mystery,Thriller",1
6441,Phantom Thread,2017,2017,130.0,"Drama,Romance",1
6487,The Shape of Water,2017,2017,123.0,"Adventure,Drama,Fantasy",2


In [4]:
get_imdb_scores(["Everything everywhere all at once", "The Fablemans"])

{'Everything everywhere all at once': 7.9, 'The Fablemans': 7.6}

titles ma genres a ratings ma imdb rating

In [ ]:
ratings = pd.read_csv('./ratings.tsv', delimiter='\t')
titles = pd.read_csv('./titles.tsv', delimiter='\t')

In [3]:
print(titles.columns)
print(ratings.columns)

Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
Index(['tconst', 'averageRating', 'numVotes'], dtype='object')


In [ ]:
ratings_df = osc_bp.merge(titles[['originalTitle', 'genres']], left_on='film', right_on='originalTitle', how='left')

use titles get genres of osc_bp

In [9]:
genres_df = osc_bp.merge(titles[['originalTitle', 'genres']], left_on='film', right_on='originalTitle', how='left')


In [11]:
genres_df.dropna(subset=['originalTitle'], inplace=True)
#filmy pojawiają się wielokrotnie z różną ilością genres - zostawiam tylko ten wiersz w którym genres jest najwięcej
genres_df['genre_count'] = genres_df['genres'].str.count(',') + 1
genres_df = genres_df.sort_values('genre_count', ascending=False).drop_duplicates('film')
genres_df.drop('genre_count', axis=1, inplace=True)
#genres_df ma 235 wierszy - osc_bp ma 591
genres_df

,year,film,wiki,winner,originalTitle,genres
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,The Broadway Melody,"Drama,Musical,Romance"
1444,1980,Raging Bull,/wiki/Raging_Bull,False,Raging Bull,"Biography,Drama,Sport"
532,1952,The Quiet Man,/wiki/The_Quiet_Man,False,The Quiet Man,"Action,Crime,Drama"
533,1953,From Here to Eternity,/wiki/From_Here_to_Eternity,True,From Here to Eternity,"Drama,Romance,War"
2074,1997,Good Will Hunting,/wiki/Good_Will_Hunting,False,Good Will Hunting,"Action,Adventure,Comedy"
...,...,...,...,...,...,...
92,1934,Viva Villa!,/wiki/Viva_Villa!,False,Viva Villa!,Western
93,1934,The White Parade,/wiki/The_White_Parade,False,The White Parade,Drama
82,1934,Flirtation Walk,/wiki/Flirtation_Walk,False,Flirtation Walk,Musical
273,1942,Kings Row,/wiki/Kings_Row,False,Kings Row,Drama


funkcja która pobiera info o genres z mojo

In [12]:
import requests
from bs4 import BeautifulSoup
import re

def scrape_genres(movie_title):
    # Build the search URL
    search_url = f'https://www.boxofficemojo.com/search/?q={movie_title}'

    # Send the search request
    search_response = requests.get(search_url)
    search_soup = BeautifulSoup(search_response.text, 'html.parser')

    # Extract the link to the movie's box office page
    movie_links = search_soup.find_all('a', class_='a-link-normal')
    if len(movie_links) == 0:
        return None
    movie_link = None
    for link in movie_links:
        if '/title/' in link['href']:
            movie_link = link
            break
    if movie_link is None:
        return None
    movie_url = f'https://www.boxofficemojo.com{movie_link["href"]}'

    # Send the box office request
    box_office_response = requests.get(movie_url)
    box_office_soup = BeautifulSoup(box_office_response.text, 'html.parser')

    genres = []
    try:
        genre_div = box_office_soup.find('div', class_='mojo-summary-values')
        if genre_div:
            genres_element = genre_div.find('span', string='Genres')
            if genres_element:
                genres = genres_element.find_next_sibling('span').text.strip()
    except (AttributeError, TypeError):
        pass
    if genres:
        genres= re.sub(r'\n\s+', ',', genres).strip(', ')
        genres= re.sub(r'\s*,\s*', ',', genres).lower()
    print(genres, movie_title)

    return genres


wybieram tytuły których genres nie ma z bazy titles - będę scrapować ich genres za pomocą powyższej funkcji

In [38]:
# Assuming 'osc_bp', 'film_values_present', and 'ratings_title' are your DataFrames
film_values = osc_bp['film'].unique()  # Use unique values from 'film' column
genres_present = genres_df[genres_df['originalTitle'].isin(film_values)]


# Create DataFrame of titles in osc_bp but not in film_values_present
genres_not_present = osc_bp[~osc_bp['film'].isin(genres_present['originalTitle'])]

print('Brak genres dla',len(genres_not_present), 'filmów')


Brak genres dla 356 filmów


In [40]:

genres_present = genres_present.drop('originalTitle', axis=1)
genres_present

,year,film,wiki,winner,genres
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,"Drama,Musical,Romance"
1444,1980,Raging Bull,/wiki/Raging_Bull,False,"Biography,Drama,Sport"
532,1952,The Quiet Man,/wiki/The_Quiet_Man,False,"Action,Crime,Drama"
533,1953,From Here to Eternity,/wiki/From_Here_to_Eternity,True,"Drama,Romance,War"
2074,1997,Good Will Hunting,/wiki/Good_Will_Hunting,False,"Action,Adventure,Comedy"
...,...,...,...,...,...
92,1934,Viva Villa!,/wiki/Viva_Villa!,False,Western
93,1934,The White Parade,/wiki/The_White_Parade,False,Drama
82,1934,Flirtation Walk,/wiki/Flirtation_Walk,False,Musical
273,1942,Kings Row,/wiki/Kings_Row,False,Drama


dodaje genres do tych których nie ma - rezultaty zapisuje od razu w scraped_results 

In [ ]:
# Create an empty DataFrame to store the scraped results
#scraped_results = pd.DataFrame(columns=['film', 'genres'])

# Iterate over each row in the DataFrame
for index, row in genres_not_present.iterrows():
    film_title = row['film']
    genres = scrape_genres(film_title)
    
    # Add the scraped results to the DataFrame
    scraped_results = scraped_results.append({'film': film_title, 'genres': genres}, ignore_index=True)
    
    # Save the results to a CSV file
    scraped_results.to_csv('genres_results.csv', index=False)



In [21]:
genres_scraped= pd.read_csv('./genres_results.csv')

In [24]:
#filmy pojawiają się wielokrotnie z różną ilością genres - zostawiam tylko ten wiersz w którym genres jest najwięcej
genres_scraped['genre_count'] = genres_scraped['genres'].str.count(',') + 1
genres_scraped = genres_scraped.sort_values('genre_count', ascending=False).drop_duplicates('film')
genres_scraped.drop('genre_count', axis=1, inplace=True)
print('znaleziono genres dla', len(genres_scraped), 'filmów z', len(genres_not_present), 'filmów którcyh brakowalo')

znaleziono genres dla 347 filmów z 356 filmów którcyh brakowalo


In [23]:
genres_scraped

,film,genres
280,The Sound of Music,"biography,drama,family,musical,romance"
185,The Letter,"crime,drama,film-noir,mystery,romance"
405,Up,"adventure,animation,comedy,drama,family"
244,Around the World in 80 Days,"action,adventure,comedy,family,western"
293,Funny Girl,"biography,comedy,drama,musical,romance"
...,...,...
162,Captains Courageous,NaN
168,Alexander's Ragtime Band,NaN
172,Pygmalion,NaN
173,Test Pilot,NaN


dodaje kolumnę genres do genres_not_present

In [35]:
# Merge the two dataframes based on the "film" column
genres_added = genres_not_present.merge(genres_scraped[['film', 'genres']], on='film', how='left')

# Drop rows with NaN values in the "genres" column
genres_added = genres_added.dropna(subset=['genres'])
# Display the updated dataframe
genres_added


,year,film,wiki,winner,genres
0,1927,Wings,/wiki/Wings_(1927_film),True,"action,drama,romance,war"
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,drama
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,"drama,fantasy,mystery,sci-fi,thriller"
3,1928,Alibi,/wiki/Alibi_(1929_film),False,"crime,thriller"
5,1928,The Patriot,/wiki/The_Patriot_(1928_film),False,"action,drama,history,war"
...,...,...,...,...,...
351,2021,The Power of the Dog,/wiki/The_Power_of_the_Dog_(film),False,"drama,romance,western"
352,2021,West Side Story,/wiki/West_Side_Story_(2021_film),False,"crime,drama,musical,romance"
353,2022,All Quiet on the Western Front,/wiki/All_Quiet_on_the_Western_Front_(2022_film),False,"drama,war"
354,2022,Elvis,/wiki/Elvis_(2022_film),False,"biography,drama,music"


In [41]:
print('genres_present: ', genres_present.columns)
print('genres_added: ', genres_added.columns)

genres_present:  Index(['year', 'film', 'wiki', 'winner', 'genres'], dtype='object')
genres_added:  Index(['year', 'film', 'wiki', 'winner', 'genres'], dtype='object')


łączę genres scraped z tymi z bazy danych 

In [61]:
# Merge the two dataframes
genres_full = pd.concat([genres_present, genres_added])

# Sort by the "year" column
genres_full = genres_full.sort_values(by='year')

# Save the merged and sorted dataframe to a CSV file
genres_full.to_csv('genres_full.csv', index=False)

genres_full


,year,film,wiki,winner,genres
0,1927,Wings,/wiki/Wings_(1927_film),True,"action,drama,romance,war"
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False,drama
2,1927,The Racket,/wiki/The_Racket_(1928_film),False,"drama,fantasy,mystery,sci-fi,thriller"
3,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,"Drama,Musical,Romance"
7,1928,In Old Arizona,/wiki/In_Old_Arizona,False,"Short,Western"
...,...,...,...,...,...
2812,2022,The Fabelmans,/wiki/The_Fabelmans,False,"Comedy,Talk-Show"
2805,2022,The Banshees of Inisherin,/wiki/The_Banshees_of_Inisherin,False,"Comedy,Drama"
2828,2022,Triangle of Sadness,/wiki/Triangle_of_Sadness,False,"Comedy,Drama"
2800,2022,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,False,"Action,Adventure,Fantasy"


Przechodzimy do ratings - merged_df łączymy przez tconst i mamy tytuł zamiast tconst i ratings w jednym

In [53]:
# Merge dataframes on id column
merged_df = pd.merge(titles[['tconst','originalTitle']], ratings[['tconst','averageRating', 'numVotes']], on='tconst')

merged_df

,tconst,originalTitle,averageRating,numVotes
0,tt0000001,Carmencita,5.7,1967
1,tt0000002,Le clown et ses chiens,5.8,264
2,tt0000003,Pauvre Pierrot,6.5,1811
3,tt0000004,Un bon bock,5.6,178
4,tt0000005,Blacksmith Scene,6.2,2610
...,...,...,...,...
1306788,tt9916730,6 Gunn,8.3,10
1306789,tt9916766,Episode #10.15,7.0,21
1306790,tt9916778,Escape,7.2,36
1306791,tt9916840,Horrid Henry's Comic Caper,8.8,6


zostawiamy tylko te genres rows gdzie mamy większą num votes i pozbywamy sie tconst i duplikatów

In [56]:
ratings_title = merged_df.drop('tconst', axis=1)
ratings_title = ratings_title.sort_values(['originalTitle', 'numVotes'], ascending = [True, False])
ratings_title = ratings_title.drop_duplicates(subset=['originalTitle'], keep='first').reset_index(drop=True)
ratings_df = ratings_title[ratings_title['originalTitle'].isin(osc_bp['film'])]
ratings_df

,originalTitle,averageRating,numVotes
5186,127 Hours,7.5,385614
21493,A Few Good Men,7.7,272238
24662,A Letter to Three Wives,7.7,10176
29993,A Serious Man,7.0,145663
30641,A Soldier's Story,7.2,10773
...,...,...,...
949708,Watch on the Rhine,7.1,4699
968349,Winter's Bone,7.1,145733
971786,Working Girl,6.8,59500
976900,Yankee Doodle Dandy,7.6,16184


mamy ratings dla 234 rows

In [ ]:
#Assuming 'osc_bp', 'film_values_present', and 'ratings_title' are your DataFrames
film_values = osc_bp['film'].unique()  # Use unique values from 'film' column
ratings_present = ratings_df.copy()


# Create DataFrame of titles in osc_bp but not in film_values_present
ratings_not_present = osc_bp[~osc_bp['film'].isin(ratings_present['originalTitle'])]

print('Brak ratings dla',len(ratings_not_present), 'filmów')


Brak ratings dla 357 filmów


In [ ]:
ratings_present.to_csv('../data/genre_bo/ratings_present.csv')
ratings_not_present.to_csv('./ratings_not_present.csv')

In [ ]:
def get_imdb_scores(movie_titles):
    """
    Given a list of movie titles, returns a dictionary of IMDb scores
    for each movie.
    """
    ia = imdb.IMDb()
    scores = {}
    for title in movie_titles:
        # search for the movie by title
        results = ia.search_movie(title)
        if results:
            # use the first search result to retrieve the IMDb score
            movie = results[0]
            ia.update(movie)
            score = movie.get('rating')
            scores[title] = score
        else:
            # if no search results are found, add an entry with None as the score
            scores[title] = None
    return scores

In [62]:
ratings_not_present

,year,film,wiki,winner
0,1927,Wings,/wiki/Wings_(1927_film),True
1,1927,7th Heaven,/wiki/7th_Heaven_(1927_film),False
2,1927,The Racket,/wiki/The_Racket_(1928_film),False
4,1928,Alibi,/wiki/Alibi_(1929_film),False
5,1928,Hollywood Revue,/wiki/Hollywood_Revue,False
...,...,...,...,...
579,2021,The Power of the Dog,/wiki/The_Power_of_the_Dog_(film),False
580,2021,West Side Story,/wiki/West_Side_Story_(2021_film),False
582,2022,All Quiet on the Western Front,/wiki/All_Quiet_on_the_Western_Front_(2022_film),False
585,2022,Elvis,/wiki/Elvis_(2022_film),False


In [ ]:
scraped_results = pd.DataFrame(columns=['film', 'rating'])
# Iterate over each row in the DataFrame
for index, row in ratings_not_present.iterrows():
    film_title = row['film']
    rating = get_imdb_scores(film_title)
    
    # Add the scraped results to the DataFrame
    scraped_results = scraped_results.append({'film': film_title, 'rating': rating}, ignore_index=True)
    
    # Save the results to a CSV file
    scraped_results.to_csv('ratings_results.csv', index=False)


In [139]:
merged_bp.to_csv('../data/genre_bo/genre_rating_num.csv')

In [25]:
genre_dummies = merged_bp['genres'].str.get_dummies(sep=',')
merged_bp = pd.concat([merged_bp[['year', 'film', 'wiki', 'winner', 'genres', 'averageRating',
       'numVotes']], genre_dummies], axis=1)

In [26]:
merged_bp = merged_bp.drop('genres', axis=1)
merged_bp.head()

,year,film,wiki,winner,averageRating,numVotes,Action,Adventure,Animation,Biography,...,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
0,1928,The Broadway Melody,/wiki/The_Broadway_Melody,True,5.6,7605,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
1,1928,In Old Arizona,/wiki/In_Old_Arizona,False,5.5,1199,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1929,The Divorcee,/wiki/The_Divorcee,False,6.7,3366,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1929,The Love Parade,/wiki/The_Love_Parade,False,7.0,2469,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
4,1931,Five Star Final,/wiki/Five_Star_Final,False,7.3,2346,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


on merged bp i apply the rating finding function - i have two df genres df with 234 rows and bo df with 200 rows and merged_df with all info 200 rows

now adding box office

funkcja która pobiera z mojo info o box office

In [38]:
import requests
from bs4 import BeautifulSoup

def scrape_box_office_info(movie_title):
    # Build the search URL
    search_url = f'https://www.boxofficemojo.com/search/?q={movie_title}'
    
    # Send the search request
    search_response = requests.get(search_url)
    search_soup = BeautifulSoup(search_response.text, 'html.parser')
    
    # Extract the link to the movie's box office page
    movie_links = search_soup.find_all('a', class_='a-link-normal')
    if len(movie_links) == 0:
        print(f'No box office information found for {movie_title}')
        return None
    movie_link = None
    for link in movie_links:
        if '/title/' in link['href']:
            movie_link = link
            break
    if movie_link is None:
        print(f'No box office information found for {movie_title}')
        return None
    movie_url = f'https://www.boxofficemojo.com{movie_link["href"]}'
    # Send the box office request
    box_office_response = requests.get(movie_url)
    box_office_soup = BeautifulSoup(box_office_response.text, 'html.parser')
   

    box_office = {'Domestic': 'N/A', 'International': 'N/A', 'Worldwide': 'N/A'}
    try:
        table = box_office_soup.find('div', class_='a-section a-spacing-none mojo-performance-summary-table')
        rows = table.find_all('div', class_='a-section a-spacing-none')
        for row in rows:
            span = row.find('span', class_='a-size-small')
            if span:
                title = span.text.strip()
                money = row.find('span', class_='money')
                if money:
                    box_office[title.split()[0]] = money.text.strip()
                else:
                    box_office[title.split()[0]] = 'N/A'
    except (AttributeError, TypeError):
        pass
    print(box_office, movie_title)
    
    return box_office

In [ ]:
bo_df = osc_bp.copy()
# Apply the function to each row in the "film" column
bo_df['box_office'] = bo_df['film'].apply(scrape_box_office_info)



In [99]:
#Usunięcie NaN 
bo_df = bo_df.dropna(subset=['box_office'])
# Sprawdzenie liczby duplikatów w kolumnie 'film'
num_duplicates = bo_df['film'].duplicated().sum()
bo_df = bo_df.drop_duplicates(subset=['film'])
# Wyświetlenie liczby duplikatów
print("Usunięto: ", num_duplicates, ' duplikatów')

In [112]:
bo_df.to_csv('../data/genre_bo/bo_full.csv')

In [ ]:
import ast

# Definicja funkcji sprawdzającej strukturę wartości box_office
def check_structure(value):
    expected_structure = {'Domestic': None, 'International': None, 'Worldwide': None}
    return isinstance(value, dict) and value.keys() == expected_structure.keys()

# Convert string values to dictionaries
def parse_box_office(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value.replace("'", "\""))
        except (ValueError, SyntaxError):
            return None
    return value

bo_cleaned = bo_df.copy()

bo_cleaned['box_office'] = bo_cleaned['box_office'].apply(parse_box_office)


# Wyświetlenie wierszy, w których struktura wartości box_office jest inna niż oczekiwana
mismatched_rows = df_cleaned[~df_cleaned['box_office'].apply(check_structure)]

# Wyświetlenie wyniku
print('obecnie ' len(mismatched_rows), 'posiada niepoprawną strukturę danych')


In [137]:
bo_final=bo_cleaned.copy()
# Create separate columns for each box office category
bo_final[['domestic_box_office', 'international_box_office', 'worldwide_box_office']] = bo_final['box_office'].apply(pd.Series)

In [138]:
bo_final.to_csv('../data/genre_bo/bo_final.csv')